# Plotting VERB Code Results

This notebook demonstrates how to visualize results from the CCMC VERB code. It includes several figures to illustrate the data generated by the VERB model.

### Initializing Kamodo Object

This section initializes the Kamodo object, which interfaces with the VERB-3D model dataset.


In [ ]:
import numpy as np
import scipy.io
import os.path
import datetime
import kamodo_ccmc.flythrough.model_wrapper as MW
from kamodo_ccmc.flythrough import SatelliteFlythrough as SF

In [ ]:
model = 'VERB-3D'  # Specify the model to be used
verb_dataset = r"C:\work\Codes\CCMC\simulations_CCMC\Alexander_Drozdov_060322_IM_4\Output\\"  # Path to the VERB dataset
cache_dataset = r"C:\work\Codes\CCMC\ccmc-flyby\cache"  # Path to the cached dataset

# Initialize the Model_Reader for the VERB-3D model
reader = MW.Model_Reader(model)

# Initialize Kamodo with the requested variables
kamodo_object = reader(verb_dataset, variables_requested=['flux_lea', 'PSD_lmk', 'kp', 'bf', 'Lpp'])

### Initializing Plotly Renderer and Importing Plotting Functions

This section initializes the Plotly renderer and imports the necessary plotting functions. Note that Plotly has known issues with LaTeX rendering, as outlined in [this GitHub issue](https://github.com/plotly/plotly.py/issues/4336). 

To work around this, we set the renderer to `iframe+png` for IDEs like PyCharm, which allows figures to render correctly without interactive features. When using a browser, `iframe` should be the default renderer for interactive plots. This setup ensures proper display until the Plotly issue is resolved.


In [ ]:
# Use Kamodo builtin plotting
from kamodo_ccmc.tools.plotfunctions import toColor, figMods
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'iframe+png'
# pio.renderers.default = 'plotly_mimetype+notebook_connected'

**Plotting 1d variables (Basic)**

This section demonstrates a simple way to plot multiple 1D variables (`bf`, `kp`, and `Lpp`) as a function of time.

In [ ]:
fig = kamodo_object.plot('bf_ijk', 'kp_ijk', 'Lpp_ijk')
fig.show()

**Plotting 1d variables (Advanced)**

In this advanced section, we create subplots to visualize the `bf`, `kp`, and `Lpp` variables individually, but all within a single figure. The subplots are stacked vertically for comparison, and each variable is plotted as a function of time.

In [ ]:
from plotly.subplots import make_subplots
fig = make_subplots(rows=3, cols=1, subplot_titles=('bf', 'kp', 'Lpp'))
fig.add_trace(kamodo_object.plot('bf_ijk').data[0], row=1, col=1)
fig.add_trace(kamodo_object.plot('kp_ijk').data[0], row=2, col=1)
fig.add_trace(kamodo_object.plot('Lpp_ijk').data[0], row=3, col=1)
fig.update_layout(xaxis3=dict(title='Time, h'))
fig.show()

**Plot L vs Time Electron Flux (Basic)**

This section demonstrates how to plot the electron flux as a function of L vs time.

In [ ]:
target_energy = 1.
target_alpha = 80.

fig = kamodo_object.plot('flux_lea_ijk', plot_partial={'flux_lea_ijk': {'E_e': target_energy, 'alpha_e' : target_alpha}})
fig.data[0].update(zauto=False, zmin=0)
fig.data[0].colorbar.title.side = 'right'
fig.update_layout(width=800)
figMods(fig, colorscale="Rainbow", log10=True, ncont=128, newTitle=f'VERB code flux. Energy: {target_energy} MeV, Pitch angle: {target_alpha} deg')
fig.show()

**L vs Time Electron Flux Plot with Comparison of Additional Variables (Advanced)**

In this section, we create a  figure that stacks multiple plots. The primary plot shows the flux as a function of L-shell and time at a specified energy and pitch angle. Below this, we add two additional plots to compare the flux data with the `kp` index and the magnetic field `bf`.

In [ ]:
target_energy = 1.
target_alpha = 80.

fig0 = kamodo_object.plot('flux_lea_ijk', plot_partial={'flux_lea_ijk': {'E_e': target_energy, 'alpha_e' : target_alpha}})
fig0.data[0].update(zauto=False, zmin=0)
fig0.data[0].colorbar.title.side = 'right'
figMods(fig0, colorscale="Rainbow", log10=True, ncont=128)

fig1 = kamodo_object.plot('kp_ijk')
fig2 = kamodo_object.plot('bf_ijk')

fig = make_subplots(rows=3, cols=1, row_heights=[0.6, 0.2, 0.2])
fig.add_trace(fig0.data[0], row=1, col=1)
fig.add_trace(fig1.data[0], row=2, col=1)
fig.add_trace(fig2.data[0], row=3, col=1)
fig.update_layout(width=800, 
                  legend=dict(yanchor="bottom",xanchor="left", x=0.85, y=0.01), 
                  title=dict(text=f'VERB code flux. Energy: {target_energy} MeV, Pitch angle: {target_alpha} deg'),
                  margin=dict(l=40, r=40, t=40, b=40),
                  yaxis=dict(title=fig0.layout.yaxis.title.text),
                  yaxis2=dict(title='kp'),
                  yaxis3=dict(title='bf'),
                  xaxis3=dict(title=fig0.layout.xaxis.title.text)
                  )
fig.show()

**L vs time PSD plot** 

This section generates a plot showing the phase space density (PSD) as a function of L-shell and time. The plot is constructed by extracting PSD data for a specified `mu` and `K`.

In [ ]:
target_mu = 700.
target_K = 0.11

fig = kamodo_object.plot('PSD_lmk_ijk', plot_partial={'PSD_lmk_ijk': {'mu': target_mu, 'K' : target_K}})
fig.data[0].update(zauto=False, zmin=-7)
fig.data[0].colorbar.title.side = 'right'
fig.update_layout(width=800)
figMods(fig, colorscale="Rainbow", log10=True, ncont=128, newTitle=f'VERB code PSD. mu: {target_mu} MeV/G, K: {target_K} R_E*G^1/2')
fig.show()

### Flux Spectrum

This section visualizes the energy spectrum of particle flux at a fixed time (in hours) and pitch angle. Note, that this plot shows interpolation artifacts due to the nature of the VERB code energy grid, which varies with L-shell. Kamodo uses nearest-neighbor interpolation to approximate the flux near the boundary of the grid.  However, far from the grid boundaries, the interpolation produces `NaN` values, in the regions where no meaningful interpolation can be performed.

In [ ]:
target_time = 45.
target_alpha = 80.

fig = kamodo_object.plot('flux_lea_ijk', plot_partial={'flux_lea_ijk': {'time': target_time, 'alpha_e' : target_alpha}})
fig.data[0].update(zauto=False, zmin=-3, y=np.log10(fig.data[0].y))
fig.data[0].colorbar.title.side = 'right'
figMods(fig, colorscale="Rainbow", 
        log10=True,
        ncont=128, 
        newTitle=f'VERB code spectrum. Time: {target_time} h, Pitch angle: {target_alpha} deg')
fig.update_layout(width=800, yaxis=dict(title='log10(E)[MeV]'))
fig.show()

**Plotting Flux as a function of energy vs pitch angle** 

This section generates a plot that visualizes the election flux as a function of energy and pitch angle at a fixed time. The Kamodo uses a gridded interpolation which also can lead to artifacts. 


In [ ]:
target_time = 45.
target_L = 4.

fig = kamodo_object.plot('flux_lea_ijk', plot_partial={'flux_lea_ijk': {'time': target_time, 'L' : target_L}})
fig.data[0].update(zauto=False, zmin=-3, x=np.log10(fig.data[0].x))
fig.data[0].colorbar.title.side = 'right'
figMods(fig, colorscale="Rainbow", 
        log10=True,
        ncont=128,
        newTitle=f'VERB code distribution. Time: {target_time} h, L: {target_L}')
fig.update_layout(width=800, xaxis=dict(title='log10(E)[MeV]'))
fig.show()

**Flyby (Basic)**

This section simulates a synthetic satellite flyby using a sine wave to generate mock L-star values that oscillate between 1 and 5.5, simulating the satellite's trajectory. The code creates timestamps at 10-minute intervals between a start and end date of the simulation. The pitch angle and energy are kept constant throughout the flyby.

The synthetic data is then used to perform a flyby via VERB code simulation, and the resulting particle flux values are functionalized into a Kamodo object for plotting.

In [ ]:
target_en = 1.
target_al = 80.
start_date, end_date = MW.File_Times(model, verb_dataset, print_output=False)
# Assume sin function of and orbit

# Timestep of 10 minutes
time_step_minutes = 10
time_steps = int((end_date - start_date).total_seconds() / (time_step_minutes * 60))

# Generate synthetic time data with 10-minute intervals
timestamps = np.array([start_date + datetime.timedelta(minutes=10 * i) for i in range(time_steps)])
timestamps = np.array([ts.timestamp() for ts in timestamps])

# Period of 9 hours for the sine wave (Lstar), oscillating between 1 and 5.5
period_seconds = 9 * 60 * 60
Lstar = np.sin(2 * np.pi * (timestamps - timestamps[0]) / period_seconds) * 2.25 + 3.25  # Oscillating between 1 and 5.5

# Keep pitch angle (a_eq) and energy (en) constant
a_eq = np.full_like(Lstar, target_al)  # Constant pitch angle
en = np.full_like(Lstar, target_en)  # Constant Energy

# Perform the flyby with synthetic data
results = SF.ModelFlythrough(model, verb_dataset, ['flux_lea'], timestamps, Lstar, en, a_eq, 'LEA-rb')

# Functionalize the output.
kamodo_object_fly = SF.O.Functionalize_SFResults(model, results)
kamodo_object_fly

In [ ]:
# Plot
kamodo_object_fly.plot('flux_lea')

**Flyby (Advanced)**

This section performs an advanced flyby simulation using real satellite data. The simulation involves loading cached data files for the RBSP mission, specifically for the selected satellite (`rbspa`). The cached files contain time, pitch angle (`a_eq`), and L-star values for each day.

- **Data Loading:** The code iterates through the dates in the specified time range, loading and concatenating `.mat` files containing the relevant fields (`time`, `a_eq`, `Lstar`). Missing files are skipped.
- **MATLAB Time Conversion:** The loaded MATLAB datenum values are converted into Python `datetime` objects.
- **Filtering:** The time data is filtered to include only the values that fall between `start_date` and `end_date`.
- **Flyby Execution:** The time, L-star, and pitch angle data, and a constant target energy of **1 MeV** are used to perform the flyby simulation. The simulation calculates the flux (`flux_lea`) along the trajectory using the VERB code simulation.
- **Functionalization:** The results of the flyby simulation are converted into a Kamodo object.
- **Plotting:** The resulting flux data is plotted, and a logarithmic scale is applied to the y-axis. The y-axis range is manually adjusted to focus on the electron flux within the heart of the radiation belts, by limiting very small values.

In [ ]:
# Variables for the flyby 
mission = 'rbsp'
sattelite = 'rbspa'
target_en = 1.

In [ ]:
# Load cached rbsp data files
start_date, end_date = MW.File_Times(model, verb_dataset, print_output=False)

fields =['time', 'a_eq', 'Lstar']

# Initialize a dictionary to store combined arrays
data = {field: [] for field in fields}

# Cycle through full days
current_date = start_date
delta = datetime.timedelta(days=1)
while current_date.date() < end_date.date():
    current_date.replace(hour=0, minute=0, second=0, microsecond=0)

    # Extract year, month, and day, and format them into a string YYMMDD
    date_str = current_date.strftime('%y%m%d')        
    mat_filename = f'{date_str}_{sattelite}_pa90_mfm501440.mat'
    mat_filepath = os.path.join(cache_dataset, mission, sattelite, f"{current_date.year % 100:02d}", f"{current_date.month:02d}", mat_filename)
    
    # Check if the file exists to avoid potential errors
    if os.path.exists(mat_filepath):
        # Load cached L* files
        mat_data = scipy.io.loadmat(mat_filepath)
        
        # Extract and append the relevant fields to the data dictionary
        for field in fields:
            if field in mat_data:
                data[field].append(mat_data[field].ravel())  # Convert to 1D array
    else:
        print(f"File not found: {mat_filepath}")
    
    current_date += delta

# After all data has been collected, concatenate the lists into NumPy arrays
for field in fields:
    data[field] = np.concatenate(data[field])
    
def matlab_datenum_to_datetime(matlab_datenum):
    """
    Convert MATLAB datenum to Python datetime.
    
    Parameters:
    - matlab_datenum: numpy array of MATLAB datenum values
    
    Returns:
    - list of Python datetime objects
    """
    # MATLAB's datenum reference date: January 1, 0000
    # Python's datetime reference date: January 1, 1970 (Unix epoch)
    datenum_offset = 719529
    days_since_0000 = matlab_datenum - datenum_offset  # Difference in days
    
    # Convert days to datetime
    return np.array([datetime.datetime(1970, 1, 1, tzinfo=datetime.timezone.utc) + datetime.timedelta(days=days) for days in days_since_0000])

data['time'] = matlab_datenum_to_datetime(data['time'])

In [ ]:
 # Create a mask where time is strictly between start_date and end_date
mask = (data['time'] >= start_date) & (data['time'] <= end_date)

# Filter the time and other data arrays
filtered_data = {key: value[mask] for key, value in data.items()}

# Convert each datetime object to a timestamp using a list comprehension
timestamps = np.array([dt.timestamp() for dt in filtered_data['time']])

# Perform flyby 
results = SF.ModelFlythrough(model, verb_dataset, ['flux_lea'], timestamps, filtered_data['Lstar'], np.full_like(filtered_data['Lstar'], target_en), filtered_data['a_eq'], 'LEA-rb')

In [ ]:
# Functionalize the output.
kamodo_object_fly = SF.O.Functionalize_SFResults(model, results)
kamodo_object_fly

In [ ]:
fig = kamodo_object_fly.plot('flux_lea')
fig.show()

fig.data[0].update(y=np.log10(fig.data[0].y))
fig.update_layout(yaxis=dict(range=[0, 5]))
fig.show()